<a href="https://colab.research.google.com/github/YiFeng/Other/blob/main/yuwei.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pip install pyddm

In [2]:
## This script was revised by Yi Feng at 07/15/2021
# Import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import walk

from ddm import Sample
import ddm.models
import ddm.plot
from ddm import Model, Fittable
from ddm.functions import fit_adjust_model, display_model
from ddm.models import NoiseConstant, BoundConstant, OverlayChain, OverlayNonDecision, OverlayPoissonMixture

In [3]:
# set work directory
data_dir = '/content/drive/MyDrive/yuwei/' #put your data folder path here
pic_dir = '/content/drive/MyDrive/pic/' #put your pic folder path here
result_dir = '/content/drive/MyDrive/' #put the folder path you want to save params here
# get all csv filenames from dir
filenames = next(walk(data_dir), (None, None, []))[2] 

In [ ]:
# create a dataframe to save params
result = pd.DataFrame(columns = ['Subid','DriftRatio','B','Nondectime'])

# read each file and run the ddm 
for each_file in filenames:
  with open(data_dir + each_file, "r") as f:
    df_rt = pd.read_csv(f)
  # delete ".csv" to extract sub id
  each_id = each_file[:len(each_file) - 4]
  
  # df_rt = df_rt[df_rt["monkey"] == 1] # Only monkey 1
    
  # Remove short and long RTs This is not strictly necessary, but is performed here for
  # compatibility with this study.
  df_rt = df_rt[df_rt["RT"] > .2] # Remove trials less than 200ms
  df_rt = df_rt[df_rt["RT"] < 2.5] # Remove trials greater than 3000ms
    
  # Create a sample object from our data.  This is the standard input
  # format for fitting procedures.  Since RT and correct/error are
  # both mandatory columns, their names are specified by command line
  # arguments.
  roitman_sample = Sample.from_pandas_dataframe(df_rt, rt_column_name="RT", correct_column_name="ACC")
  class DriftRatio(ddm.models.Drift):
      name = "Drift depends linearly on Ratio"
      required_parameters = ["driftRatio"] # <-- Parameters we want to include in the model
      required_conditions = ["Ratio"] # <-- Task parameters ("conditions"). Should be the same name as in the sample.
      
      # We must always define the get_drift function, which is used to compute the instantaneous value of drift.
      def get_drift(self, conditions, **kwargs):
          return self.driftRatio * conditions['Ratio']

  model_rs = Model(name='Roitman data, drift varies with Ratio',
                  drift=DriftRatio(driftRatio=Fittable(minval=0, maxval=20)),
                  noise=NoiseConstant(noise=1),
                  bound=BoundConstant(B=Fittable(minval=.1, maxval=1.5)),
                  # Since we can only have one overlay, we use
                  # OverlayChain to string together multiple overlays.
                  # They are applied sequentially in order.  OverlayNonDecision
                  # implements a non-decision time by shifting the
                  # resulting distribution of response times by
                  # `nondectime` seconds.
                  overlay=OverlayChain(overlays=[OverlayNonDecision(nondectime=Fittable(minval=0, maxval=.4)),
                                                  OverlayPoissonMixture(pmixturecoef=.02,
                                                                        rate=1)]),
                  dx=.001, dt=.01, T_dur=2)
  # Fitting this will also be fast because PyDDM can automatically
  # determine that DriftRatio will allow an analytical solution.
  fit_model_rs, x_fit = fit_adjust_model(sample=roitman_sample, model=model_rs, verbose=False)

  # display_model(fit_model_rs)
  
  # Return result
  # README: please remmember to change the source code in function.py at line365, which should be: return m, x_fit.x
  # x_fit.x is an array that contains all params: ['driftRatio','B','nondectime']
 
  # create a dictionary for each sub to link params' names and values, also adding subid
  data_to_append = {}
  data_to_append['Subid'] = each_id
  for i in range(1,len(result.columns)):
    data_to_append[result.columns[i]] = x_fit[i-1]
  # append all participants' params to dataframe
  result = result.append(data_to_append, ignore_index = True)
  
  # plot
  Model.get_fit_result(fit_model_rs)
  fig,ax = plt.subplots(figsize=(6,4)) # define fig size
  ddm.plot.plot_fit_diagnostics(model=fit_model_rs, sample=roitman_sample)
  plt.savefig(pic_dir + each_id + ".png")
  # plt.show()

# save result to csv
result.to_csv(result_dir + '/result.csv')